In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors

%matplotlib inline

pd.set_option("display.max_columns", None)

path = "C:/Users/Admin/Documents/ironhack/streaming_service_recommender/"

## Streaming Service Recommender

#### Goals

- wrap all up together
- use functions in this notebook to create a module to use for the streaming service recommender

### 1. Import data

Import all necessary data.

In [3]:
url = "https://docs.google.com/spreadsheets/d/1VsL_x7WAcEm-pLNmCqNwrDlFjTR_PUDpiadyUYJM-AQ/export?format=csv&gid=1360685521"
survey_df = pd.read_csv(url)

all_shows = pd.read_pickle(path + "Data/all_shows_genres.pkl")

genres_recommender = pd.read_pickle(path + "Data/genres_recommender.pkl")

genre_columns = [col for col in all_shows.columns if col not in ["show", "genres"]]